<a href="https://colab.research.google.com/github/kankkw/229352-StatisticalLearning/blob/main/Lab03_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 3 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้นบันทึกเป็นไฟล์ *.pdf

### ในปฏิบัติการนี้เราจะฝึกการทำ PCA ด้วยสองวิธี คือ
1. หาด้วยการแยกส่วนประกอบของเมทริกซ์ผ่าน numpy
2. หาด้วยการใช้เครื่องมือที่มีมาให้ใน scikit-learn

ดาวน์โหลดข้อมูลสัตว์ต่างๆ 50 ชนิดดังนี้  
ชนิดสัตว์: https://donlapark.pages.dev/229351/data/classes.txt  
ตัวแปรต่างๆ: https://donlapark.pages.dev/229351/data/predicates.txt  
ค่าของสัตว์แต่ละชนิด: https://donlapark.pages.dev/229351/data/predicate-matrix-continuous.txt

In [ ]:
!pip install -q wget

import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA



In [ ]:
# Download the files
!wget -O classes.txt https://donlapark.pages.dev/229351/data/classes.txt
!wget -O predicate-matrix-continuous.txt https://donlapark.pages.dev/229351/data/predicate-matrix-continuous.txt

In [ ]:
classes = np.genfromtxt('classes.txt',dtype='str')

classes[:5]

In [ ]:
data = np.genfromtxt('predicate-matrix-continuous.txt')

data[:5,:5]

#### Exercise 1

ใน code block ข้างล่างนี้ จงทำ PCA บนข้อมูลที่ได้มาให้เหลือเมทริกซ์ข้อมูลที่มีตัวแปรแค่ 2 ตัว โดยใช้ฟังก์ชัน $\texttt{np.linalg.eigh}$  
ดังนั้น เมทริกซ์ที่ได้ต้องมีขนาด 50x2

$$ \Sigma =
  \begin{pmatrix}
\text{var}(X_1) & \text{cov}(X_1,X_2) & \cdots & \text{cov}(X_1,X_{85}) \\
\text{cov}(X_2,X_1) & \text{var}(X_2) & \cdots & \text{cov}(X_2,X_{85}) \\
\vdots & \vdots & \ddots & \vdots \\
\text{cov}(X_{85},X_1) & \text{cov}(X_{85},X_1) & \cdots & \text{var}(X_{85}) \\
\end{pmatrix}
 $$

In [ ]:
import numpy as np

# 1. Center ข้อมูล
data_c = data - np.mean(data, axis=0)

# 2. หา covariance matrix
cov = np.cov(data_c.T)

# 3. Decompose the covariance matrix UDU^T
eig_vals, eig_vecs = np.linalg.eigh(cov)

# 4. เรียง eigenvectors จากค่ามากไปน้อย แล้วเลือก 2 ตัวแรก
sorted_idx = np.argsort(eig_vals)[::-1]
U = eig_vecs[:, sorted_idx[:2]]

# 5. Projection → ลดเหลือ 2 มิติ
projected = np.dot(data_c, U)

#### OPTIONAL

ใน code block ข้างล่างนี้ จงทำ PCA บนข้อมูลที่ได้มาให้เหลือเมทริกซ์ข้อมูลที่มีตัวแปรแค่ 2 ตัว โดยใช้ $scikit-learn$  
พร้อมกับตรวจสอบว่าเมทริกซ์ที่ได้จากทั้งสองวิธีนี้มีค่าเท่ากัน (ต่างกันแค่ค่าบวกลบ)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
projected_sklearn = pca.fit_transform(data)

np.allclose(np.abs(projected), np.abs(projected_sklearn))

In [ ]:
#TODO (optional): enter code here

#### Exercise 2

ทำการพล็อตจุดของข้อมูลที่ได้จาก PCA ข้างบนพร้อมกับใส่คำกำกับว่าจุดไหนเป็นของสัตว์ชนิดใดโดยใช้ชื่อสัตว์จาก classes.txt

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 16))
plt.scatter(projected[:, 0], projected[:, 1])

for i in range(50):
    plt.annotate(
        classes[i, 1],
        xy=(projected[i, 0], projected[i, 1]),
        xytext=(5, -8),
        textcoords='offset pixels'
    )

plt.title("PCA Projection of Animal Data (2D)")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.show()

#### Exercise 3

หากลุ่มสัตว์ต่างๆ ที่อยู่ใกล้กันมา 4 กลุ่ม แล้วลองอธิบายว่าสัตว์ในแต่ละกลุ่มมีอะไรที่เหมือนกัน

1. กลุ่มสัตว์ที่มุมซ้ายล่าง ตัวอย่างเช่น beaver, dalmatian, persian cat, skunk, mole มีลักษณะที่เหมือนกันคือ สัตว์ในกลุ่มนี้ส่วนใหญ่เป็นสัตว์เลี้ยงลูกด้วยนมขนาดเล็ก มีขนปุกปุย มีพฤติกรรมอาศัยอยู่บนพื้นดินหรือในรู บางชนิดสามารถเลี้ยงเป็นสัตว์เลี้ยงได้ เช่น สุนัขหรือแมว
2. กลุ่มสัตว์ที่มุมขวาบน ตัวอย่างเช่น killer whale, blue whale, humpback whale, seal, otter มีลักษณะที่เหมือนกันคือ เป็นสัตว์ทะเลหรือสัตว์เลี้ยงลูกด้วยนมที่ใช้ชีวิตในน้ำ มีความสามารถในการว่ายน้ำ รูปร่างเพรียวยาว บางชนิดมีฟลิปเปอร์และใช้คลื่นเสียงในการสื่อสารหรือหาทาง

3. กลุ่มสัตว์ที่มุมขวาล่าง ตัวอย่างเช่น grizzly bear, german shepherd, siamese cat, tiger, leopard มีลักษณะที่เหมือนกันคือ เป็นสัตว์นักล่า มีเขี้ยวและกรงเล็บแหลม หลายตัวอยู่ในวงศ์แมวหรือหมา เคลื่อนไหวว่องไวและมีพละกำลังสูง ใช้การล่าเพื่อเอาตัวรอด
4. กลุ่มสัตว์ที่มุมซ้ายบน ตัวอย่างเช่น antelope, horse, hippopotamus, moose, elephant มีลักษณะที่เหมือนกันคือ เป็นสัตว์กินพืชขนาดใหญ่ เดินสี่ขา มีร่างกายแข็งแรง อาศัยอยู่ตามทุ่งหญ้าหรือป่าเปิดโล่ง มีขนาดลำตัวใหญ่และมีอวัยวะที่เหมาะสมกับการเดินทางระยะไกล